In [1]:
from matching_functions import *

EEBO_SHINGLE_FOLDER = '/home/spenteco/0/eebo_shingled/'
METADATA_FILE = 'metadata/EEBO_metadata.tsv'
OUTPUT_FOLDER = 'text_reuse_html_outputs/'

metadata = load_metadata(METADATA_FILE)

In [2]:
for k, v in metadata.items():
    if 'Herrick' in v['author'] and v['year'] == '1648':
        print(k, v)
print()

for k, v in metadata.items():
    if 'Spenser' in v['author'] and v['year'] == '1590':
        print(k, v)
print()

for k, v in metadata.items():
    if 'Browne' in v['author'] and 'Hydriotaphia' in v['title'] and v['year'] == '1658':
        print(k, v)
print()

for k, v in metadata.items():
    if 'Burton, R' in v['author'] and 'melancholy' in v['title']:
        print(k, v)
print()

A43441 {'year': '1648', 'author': 'Herrick, Robert, 1591-1674.|Marshall, William, fl. 1617-1650.', 'title': 'Hesperides, or, The works both humane & divine of Robert Herrick, Esq.'}

A12777 {'year': '1590', 'author': 'Spenser, Edmund, 1552?-1599.', 'title': 'The faerie qveene disposed into twelue books, fashioning XII. morall vertues.'}

A77689 {'year': '1658', 'author': 'Browne, Thomas, Sir, 1605-1682.', 'title': 'Hydriotaphia, urne-buriall, or, a discourse of the sepulchrall urnes lately found in Norfolk. Together with the garden of Cyrus, or the quincunciall, lozenge, or net-work plantations of the ancients, artificially, naturally, mystically considered. With sundry observations. / By Thomas Browne D. of Physick.'}

A17310 {'year': '1621', 'author': 'Burton, Robert, 1577-1640.', 'title': 'The anatomy of melancholy vvhat it is. VVith all the kindes, causes, symptomes, prognostickes, and seuerall cures of it. In three maine partitions with their seuerall sections, members, and subsec

In [14]:
import glob, time
from mako.template import Template

MAX_GAP_ALLOWED = 5
MIN_MATCH_LENGTH = 6

def find_text_reuse(from_tcp_id):
    
    start_time = time.time()
    
    t = Template(filename='matching_results_template.html')
    
    from_author = metadata[from_tcp_id]['author']
    from_title = metadata[from_tcp_id]['title']
    from_year = metadata[from_tcp_id]['year']
    
    from_file = load_pickle_file(EEBO_SHINGLE_FOLDER + from_tcp_id + '.pickle')
    
    all_results = []
    quit_early = False
    
    for pn, p in enumerate(glob.glob(EEBO_SHINGLE_FOLDER + '*.pickle')):
        
        if quit_early == True:
            break
        
        #if pn % 1000 == 0:
        #    print('processing', pn)
        
        to_tcp_id = p.split('/')[-1].split('.')[0]
        
        if to_tcp_id == from_tcp_id:
            continue
        
        try:
    
            to_author = metadata[to_tcp_id]['author']
            to_title = metadata[to_tcp_id]['title']
            to_year = metadata[to_tcp_id]['year']

            to_file = load_pickle_file(EEBO_SHINGLE_FOLDER + to_tcp_id + '.pickle')

            results = match_two_files(from_file, 
                                        to_file,
                                        MAX_GAP_ALLOWED, MIN_MATCH_LENGTH)

            if len(results) > 0:
                all_results.append([[to_tcp_id, to_author, to_title, to_year], results])
                #quit_early = True
                
        except KeyError:
            print('ERROR -- metadata?', to_tcp_id)
            
    #print(all_results[0])
    
    f = open(OUTPUT_FOLDER + from_tcp_id + '.html', 'w', encoding='utf-8')
    f.write(t.render(from_tcp_id=from_tcp_id,
                        from_year=from_year,
                        from_author=from_author,
                        from_title=from_title,
                        all_results=all_results))
                        
    f.close()
    
    stop_time = time.time()
    
    print(from_tcp_id, 'done!', (stop_time - start_time))

In [15]:
for tcp_id in ['A43441', 'A12777', 'A77689', 'A17310']:
    find_text_reuse(tcp_id)

print('Done!')

ERROR -- metadata? A97378_test
ERROR -- metadata? A49690
ERROR -- metadata? A49004
ERROR -- metadata? A70386
ERROR -- metadata? A40724
ERROR -- metadata? A07753-001
ERROR -- metadata? A01504-001
A43441 done! 2736.2680370807648
ERROR -- metadata? A97378_test
ERROR -- metadata? A49690
ERROR -- metadata? A49004
ERROR -- metadata? A70386
ERROR -- metadata? A40724
ERROR -- metadata? A07753-001
ERROR -- metadata? A01504-001
A12777 done! 2961.65456700325
ERROR -- metadata? A97378_test
ERROR -- metadata? A49690
ERROR -- metadata? A49004
ERROR -- metadata? A70386
ERROR -- metadata? A40724
ERROR -- metadata? A07753-001
ERROR -- metadata? A01504-001
A77689 done! 2702.961324453354
ERROR -- metadata? A97378_test
ERROR -- metadata? A49690
ERROR -- metadata? A49004
ERROR -- metadata? A70386
ERROR -- metadata? A40724
ERROR -- metadata? A07753-001
ERROR -- metadata? A01504-001
A17310 done! 3662.5890316963196
Done!
